In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib as mpl
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.source_extraction.cnmf import params as params
from caiman.source_extraction.cnmf.deconvolution import constrained_foopsi
from caiman.source_extraction.cnmf.deconvolution import constrained_oasisAR2


def Okada_filter(trace):
    """Trace is a list.
    This will apply Okada filter to it as described in Ishikawa et al, STAR Protocols 2020"""
    x=trace.copy()
    M=np.mean(x)
    SD=np.std(x)
    T=len(x)
    for t in range(2, T-1):
        Z=abs(x[t+1]-M)/SD
        if (x[t]-x[t+1])*(x[t]-x[t+1])>0:
            x[t]=(x[t-1]+Z*x[t]+x[t+1])/(2+Z)
    return x

def subtract_baseline(trace, order):
    x=np.linspace(0,len(trace)*0.05,len(trace))
    p = np.poly1d(np.polyfit(x, trace, int(order)))
    return p

def dFF(trace):
    """Trace is a list.
    This will calculate deltaF/F0"""
    x=trace.copy()
    M=np.mean(trace)
    T=len(trace)
    for t in range(0, T):
        x[t]=(trace[t]-M)/M
    return x


def process(trace, polynomial_order=10, Okada_order=4):
    """Trace is a list.
    This will subtract the background with a nth order polynomial and apply Okada filter (m interactions)
    
    n=polynomial_order (default is 10)
    m=number of consecutive Okada iteraction (default is 4)
    
    """
    x=trace.tolist().copy()
    baseline=subtract_baseline(x,polynomial_order)
    
    t = np.linspace(0, len(x)*0.05,len(x))
    
    x = [a - b for a, b in zip(x, baseline(t))]

    for i in range(Okada_order):
        
        x=Okada_filter(x)
        
    return x    
   
framerate=20 #Hz for interframe time 50ms

In [ ]:
#Select PRE ROIs


folder=r'\\AA06299633\S2410250541\Results' # Input folder

file="20241210_S2410250541_field01_day1_mScarletxCre_Ch1mScarlet-Ch2GCaMP7b_PRE_Results.csv" #File location of PRE recording


df=pd.read_csv(os.path.join(folder, file))
df=df.iloc[:, 1:]

for column in df.columns:
    if column[:4]!='Mean' :
        df=df.drop(column, axis=1)
    elif column[:4]=='Mean' :
        df=df.rename(columns={column:'ROI'+column[4:]})
        
df2=df.copy()

for column in df2.columns:
    df2[column]=dFF(df2[column]) #calculates Delta F/F
    df2[column]=process(df2[column]) # subtracts background and filters noise

cells=df.columns

from matplotlib import gridspec
from sklearn.preprocessing import normalize


#plotting params
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = '0.5'
mpl.rcParams['axes.labelsize'] = '8'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['xtick.labelsize'] = '4'
mpl.rcParams['ytick.labelsize'] = '4'
mpl.rcParams['ytick.color'] = 'black'

bl=None
c1=None                               #initial calcium concentation (value)
g=None
sn=10                                 #noise SD
p=1
method_deconvolution='oasis'
bas_nonneg=True                       #baseline estimation
noise_range=[0.25, 0.75]              #frquency range to estimate noise
noise_method='logmexp'                #method to estimate noise
lags=5
fudge_factor=.99
verbosity=False
solvers=None
optimize_g=5
s_min=2.5 #if z-scored use s_min=2.5 equivalent to 2.5SD ; elsse use s_min between .1 and .25 (usually .2)

#--------------------------------------

trace_dlc = np.array(df2.values.astype(float)).transpose()[:]
cells_tv = cells[:]
all_events = []
all_events_count=[]
all_events_frequency=[]
for cell in range(len(trace_dlc)):

     raw_trace = trace_dlc[cell]
     raw_trace = np.array(raw_trace)#/max(np.array(raw_trace)) #If we want traces between 0 and 1 (OPTIONAL)
     raw_trace = (raw_trace - np.mean(raw_trace))/np.std(raw_trace) #z-score normalised

     #OASIS function run through
     events = constrained_foopsi(raw_trace,
                                 bl=bl, 
                                 c1=c1,
                                 g=g, 
                                 sn=sn,
                                 p=p,
                                 method_deconvolution=method_deconvolution,
                                 bas_nonneg=bas_nonneg,
                                 noise_range=noise_range,
                                 noise_method=noise_method,
                                 lags=lags,
                                 fudge_factor=fudge_factor,
                                 verbosity=verbosity,
                                 solvers=solvers,
                                 optimize_g=optimize_g,
                                 s_min=s_min)
                             
 
     all_events.append(events[5])
     all_events_frequency.append(np.count_nonzero(events[5])/len(events[5])*framerate)
     all_events_count.append(np.count_nonzero(events[5]))

    
    
     #Visualising data
     fig, axs = plt.subplots(3, 1, figsize=(5,1), dpi=400, facecolor='w', edgecolor='k' )
     gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1], )
     fig.suptitle(cells_tv[cell], fontsize=10)

     for ax in fig.get_axes():
         ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
    
     #Raw traces
     axs[0] = plt.subplot(gs[0])
     axs[0] = plt.plot( raw_trace, c='green', linewidth=.5)
    
     #OASIS noise consideration
     axs[1] = plt.subplot(gs[1])
     axs[1] = plt.plot(events[0], c='b', linewidth=.5)
     
     #Events plotted on raw trace
     axs[2] = plt.subplot(gs[2])
     axs[2] = plt.plot(raw_trace, c='k', alpha=0.1, linewidth=.5)
     axs[2] = plt.scatter( np.arange(0,len(events[5]),1),events[5], c='r', s=.5)
    
     fig.text(0.5, -0.1, 'Frame', ha='center', fontsize=5)
     fig.text(0.05, 0.5, u'Δ F/F (z-score)', va='center', rotation='vertical', fontsize=5)

     
     plt.subplots_adjust(wspace=0, hspace=0)
     plt.show()

df_events=pd.DataFrame(data={'ROI':cells,'PRE_n':all_events_count,'PRE_Hz':all_events_frequency})


In [ ]:
#Select POST ROIs

file="20241210_S2410250541_field01_day1_mScarletxCre_Ch1mScarlet-Ch2GCaMP7b_POST_Results.csv" #File location of POST recording


df=pd.read_csv(os.path.join(folder, file))
df=df.iloc[:, 1:]

for column in df.columns:
    if column[:4]!='Mean' :
        df=df.drop(column, axis=1)
    elif column[:4]=='Mean' :
        df=df.rename(columns={column:'ROI'+column[4:]})
        
df2=df.copy()

for column in df2.columns:
    df2[column]=dFF(df2[column]) #calculates Delta F/F
    df2[column]=process(df2[column]) # subtracts background and filters noise

cells=df.columns

from matplotlib import gridspec
from sklearn.preprocessing import normalize


#plotting params
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = '0.5'
mpl.rcParams['axes.labelsize'] = '8'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['xtick.labelsize'] = '4'
mpl.rcParams['ytick.labelsize'] = '4'
mpl.rcParams['ytick.color'] = 'black'

bl=None
c1=None                               #initial calcium concentation (value)
g=None
sn=10                                 #noise SD
p=1
method_deconvolution='oasis'
bas_nonneg=True                       #baseline estimation
noise_range=[0.25, 0.75]              #frquency range to estimate noise
noise_method='logmexp'                #method to estimate noise
lags=5
fudge_factor=.99
verbosity=False
solvers=None
optimize_g=5
s_min=2.2 #if z-scored use s_min=2 equivalent to 2SD ; elsse use s_min between .1 and .25 (usually .2)

#--------------------------------------

trace_dlc = np.array(df2.values.astype(float)).transpose()[:]
cells_tv = cells[:]
all_events = []
all_events_count=[]
all_events_frequency=[]
for cell in range(len(trace_dlc)):

     raw_trace = trace_dlc[cell]
     raw_trace = np.array(raw_trace)#/max(np.array(raw_trace)) #If we want traces between 0 and 1 (OPTIONAL)
     raw_trace = (raw_trace - np.mean(raw_trace))/np.std(raw_trace) #z-score normalised

     #OASIS function run through
     events = constrained_foopsi(raw_trace,
                                 bl=bl, 
                                 c1=c1,
                                 g=g, 
                                 sn=sn,
                                 p=p,
                                 method_deconvolution=method_deconvolution,
                                 bas_nonneg=bas_nonneg,
                                 noise_range=noise_range,
                                 noise_method=noise_method,
                                 lags=lags,
                                 fudge_factor=fudge_factor,
                                 verbosity=verbosity,
                                 solvers=solvers,
                                 optimize_g=optimize_g,
                                 s_min=s_min)
                             
 
     all_events.append(events[5])
     all_events_frequency.append(np.count_nonzero(events[5])/len(events[5])*framerate)
     all_events_count.append(np.count_nonzero(events[5]))
    
    
     #Viualising data
     fig, axs = plt.subplots(3, 1, figsize=(5,1), dpi=400, facecolor='w', edgecolor='k' )
     gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1], )
     fig.suptitle(cells_tv[cell], fontsize=10)

     for ax in fig.get_axes():
         ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
    
     #Raw traces
     axs[0] = plt.subplot(gs[0])
     axs[0] = plt.plot( raw_trace, c='green', linewidth=.5)
    
     #OASIS noise consideration
     axs[1] = plt.subplot(gs[1])
     axs[1] = plt.plot(events[0], c='b', linewidth=.5)
     
     #Events plotted on raw trace
     axs[2] = plt.subplot(gs[2])
     axs[2] = plt.plot(raw_trace, c='k', alpha=0.1, linewidth=.5)
     axs[2] = plt.scatter( np.arange(0,len(events[5]),1),events[5], c='r', s=.5)
    
     fig.text(0.5, -0.1, 'Frame', ha='center', fontsize=5)
     fig.text(0.05, 0.5, u'Δ F/F (z-score)', va='center', rotation='vertical', fontsize=5)

     
     plt.subplots_adjust(wspace=0, hspace=0)
     plt.show()

df_events_post=pd.DataFrame(data={'ROI':cells,'POST_n':all_events_count,'POST_Hz':all_events_frequency})

df_events=pd.merge(df_events, df_events_post, on='ROI')

In [ ]:
#Select POST ROIs

file="20241210_S2410250541_field01_day1_mScarletxCre_Ch1mScarlet-Ch2GCaMP7b_POST2_Results.csv" #Optional :File location of second POST recording


df=pd.read_csv(os.path.join(folder, file))
df=df.iloc[:, 1:]

for column in df.columns:
    if column[:4]!='Mean' :
        df=df.drop(column, axis=1)
    elif column[:4]=='Mean' :
        df=df.rename(columns={column:'ROI'+column[4:]})
        
df2=df.copy()

for column in df2.columns:
    df2[column]=dFF(df2[column]) #calculates Delta F/F
    df2[column]=process(df2[column]) # subtracts background and filters noise

cells=df.columns

from matplotlib import gridspec
from sklearn.preprocessing import normalize


#plotting params
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = '0.5'
mpl.rcParams['axes.labelsize'] = '8'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['xtick.labelsize'] = '4'
mpl.rcParams['ytick.labelsize'] = '4'
mpl.rcParams['ytick.color'] = 'black'

bl=None
c1=None                               #initial calcium concentation (value)
g=None
sn=10                                 #noise SD
p=1
method_deconvolution='oasis'
bas_nonneg=True                       #baseline estimation
noise_range=[0.25, 0.75]              #frquency range to estimate noise
noise_method='logmexp'                #method to estimate noise
lags=5
fudge_factor=.99
verbosity=False
solvers=None
optimize_g=5
s_min=2.2 #if z-scored use s_min=2 equivalent to 2SD ; elsse use s_min between .1 and .25 (usually .2)

#--------------------------------------

trace_dlc = np.array(df2.values.astype(float)).transpose()[:]
cells_tv = cells[:]
all_events = []
all_events_count=[]
all_events_frequency=[]
for cell in range(len(trace_dlc)):

     raw_trace = trace_dlc[cell]
     raw_trace = np.array(raw_trace)#/max(np.array(raw_trace)) #If we want traces between 0 and 1 (OPTIONAL)
     raw_trace = (raw_trace - np.mean(raw_trace))/np.std(raw_trace) #z-score normalised

     #OASIS function run through
     events = constrained_foopsi(raw_trace,
                                 bl=bl, 
                                 c1=c1,
                                 g=g, 
                                 sn=sn,
                                 p=p,
                                 method_deconvolution=method_deconvolution,
                                 bas_nonneg=bas_nonneg,
                                 noise_range=noise_range,
                                 noise_method=noise_method,
                                 lags=lags,
                                 fudge_factor=fudge_factor,
                                 verbosity=verbosity,
                                 solvers=solvers,
                                 optimize_g=optimize_g,
                                 s_min=s_min)
                             
 
     all_events.append(events[5])
     all_events_frequency.append(np.count_nonzero(events[5])/len(events[5])*framerate)
     all_events_count.append(np.count_nonzero(events[5]))
    
    
     #Viualising data
     fig, axs = plt.subplots(3, 1, figsize=(5,1), dpi=400, facecolor='w', edgecolor='k' )
     gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1], )
     fig.suptitle(cells_tv[cell], fontsize=10)

     for ax in fig.get_axes():
         ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
    
     #Raw traces
     axs[0] = plt.subplot(gs[0])
     axs[0] = plt.plot( raw_trace, c='green', linewidth=.5)
    
     #OASIS noise consideration
     axs[1] = plt.subplot(gs[1])
     axs[1] = plt.plot(events[0], c='b', linewidth=.5)
     
     #Events plotted on raw trace
     axs[2] = plt.subplot(gs[2])
     axs[2] = plt.plot(raw_trace, c='k', alpha=0.1, linewidth=.5)
     axs[2] = plt.scatter( np.arange(0,len(events[5]),1),events[5], c='r', s=.5)
    
     fig.text(0.5, -0.1, 'Frame', ha='center', fontsize=5)
     fig.text(0.05, 0.5, u'Δ F/F (z-score)', va='center', rotation='vertical', fontsize=5)

     
     plt.subplots_adjust(wspace=0, hspace=0)
     plt.show()

df_events_post=pd.DataFrame(data={'ROI':cells,'POST2_n':all_events_count,'POST2_Hz':all_events_frequency})

df_events=pd.merge(df_events, df_events_post, on='ROI')

In [ ]:
df_events

In [ ]:
df_events.to_csv(os.path.join(folder, file[:21]+'EVE.csv'))